# Vegetation emissions from raster map

Here we produce emissions using data coming from a raster

I originally tried to produce the data on a 1m grid, but GRAL ran out of memory when doing that. So I have reduced the resolution to 5m.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path


import rioxarray as rxr
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, Polygon

from pygg.grids import GralGrid
from emiproc.exports.gral import export_to_gral

from emiproc.inventories import EmissionInfo

In [ ]:
crs = "LV03"

landuse_raster=Path("/scratch/snx3000/lconstan/data/zurich_landuse_1m.tif")
gral_dir = Path("/store/empa/em05/isuter/projects/Zurich_CO2_clean/")
tree_heights_file = Path("/scratch/snx3000/lconstan/data/Vegetationshoehenmodell_2019_1m_2056_zurich.tif")
output_dir = Path("/scratch/snx3000/lconstan/data/vegetation_emissons_10m_groundandheight_landusemap")
output_dir.mkdir(exist_ok=True)
Land_cover_to_VPRM_class={
    1: 'Evergreen',
    2: 'Evergreen',
    3: 'Deciduous',
    4: 'Deciduous',
    5: 'Mixed',
    
    10: 'Deciduous',
    20: 'Shrubland',
    30: 'Grassland',
    40: 'Cropland',
    50: 'Others',
    60: 'Savanna', 
    70: 'Others',
    80: 'Others',
    90: 'Others',
}
sg_of_cat = {
    'Evergreen': 50 ,
    'Deciduous': 51 ,
    'Mixed': 52 ,
    'Grassland': 53 ,
    'Cropland': 54 ,
    'Evergreen_ground': 55 ,
    'Deciduous_ground': 56 ,
    'Mixed_ground': 57 ,
    'Grassland_ground': 58 ,
    'Cropland_ground': 59 ,
}
classes_to_convert = list(sg_of_cat.keys())
land_cover_values_of_class = {
    c : [k for k,v in Land_cover_to_VPRM_class.items() if v==c] for c in Land_cover_to_VPRM_class.values() if c in classes_to_convert
}
land_cover_values_of_class

In [ ]:



grid = GralGrid.from_gral_rundir(gral_dir)
grid.crs = crs

gral_gdf = gpd.GeoDataFrame(
    geometry=[grid.get_bounding_polygon()],
    crs=grid.crs
).to_crs('LV95')



In [ ]:
veg_height_da = rxr.open_rasterio(tree_heights_file).squeeze()
# clip to gral
veg_height_da = veg_height_da.rio.clip_box(*gral_gdf.total_bounds).squeeze()
veg_height_da

In [ ]:
from rasterio.enums import Resampling
downscale_factor = 10
new_width = int(veg_height_da.rio.width / downscale_factor)
new_height = int(veg_height_da.rio.height / downscale_factor)

veg_downsample_da = veg_height_da.rio.reproject(
    veg_height_da.rio.crs,
    shape=(new_height, new_width),
    resampling=Resampling.average,
)



In [ ]:
landuse_da = rxr.open_rasterio(landuse_raster).squeeze()
# Crop the landuse raster to the bounding box of the gral grid
landuse_da = landuse_da.rio.clip_box(*gral_gdf.total_bounds)


landuse_da

In [ ]:
masks = {}
for c, values in land_cover_values_of_class.items():

    mask = landuse_da.isin(values)
    # Add the class as a new dimension
    mask = mask.expand_dims('class')
    # Rename the dimension to class
    mask = mask.rename({'band': 'class'})
    # Set the class value
    mask['class'] = [c]

    masks[c] = mask

In [ ]:
ds_classes = xr.concat(masks.values(), dim='class')

new_width = int(ds_classes.rio.width / downscale_factor)
new_height = int(ds_classes.rio.height / downscale_factor)

ds_classes_resampled = ds_classes.astype(float).rio.reproject(
    veg_height_da.rio.crs,
    shape=(new_height, new_width),
    resampling=Resampling.average,
)

In [ ]:
gral_gdf.total_bounds

In [ ]:
dfs = {}
# xmin, ymin, xmax, ymax = gral_gdf.total_bounds
for cat, map in ds_classes_resampled.groupby("class"):
    map = map.squeeze()
    # extract the coordinates of the cells
    x, y = np.where(map > 0.02)
    z = np.round(veg_downsample_da.values[x, y], 1)
    lon, lat = map.x.values[y], map.y.values[x]
    emission_rate = np.round(map.values[x, y], 2)
    # Convert to the crs of the simulation
    dx, dy = map.rio.resolution()
    cell_start = gpd.points_from_xy(lon - dx / 2, lat - dy / 2, crs=map.rio.crs).to_crs(
        "LV03"
    )
    lon, lat
    # Create squares for each cell
    # Save the squares
    for t in ['normal', 'ground']:
        cat = cat if t=='normal' else f'{cat}_{t}'
        # We assume the emission go from the half size to the top
        z_clipped = np.maximum(z / 2.0, 0.1)
        z_to_write = z_clipped if t=='normal' else np.full_like(z_clipped, 0.1)
        dfs[cat] = pd.DataFrame(
            {
                "x": np.round(cell_start.x, 1),
                "y": np.round(cell_start.y, 1),
                "z":  np.round(z_to_write, 1),
                "dx": np.round(dx, 0),
                "dy": np.round(np.abs(dy), 0),
                "dz": np.round(z_to_write, 1),
                "emisson_rate": emission_rate,
                "u1": 0,
                "u2": 0,
                "u3": 0,
                "category": sg_of_cat[cat],
            }
        )

In [ ]:
with open(output_dir / "cadastre.dat", "w") as f:
    # 1 line ignored
    header = "x,y,z,dx,dy,dz,emission_rate[kg/h],-,-,-,source_group\n"
    f.write(header)
    pd.concat(dfs.values()).to_csv(f, index=False, header=False, sep=",")


In [ ]:
# SAve the source group in a json file
import json
with open(output_dir / "source_groups.json", "w") as f:
    json.dump({sg: cat for cat, sg in sg_of_cat.items()}, f, indent=4)

In [ ]:
# Read the output file of gral 

df = pd.read_csv("/scratch/snx3000/lconstan/gral/vegetation_zh/run_dir/GRAL_00001/ReceptorConcentrations.dat", sep="\t", header=[0,1], 
            )
df.drop(df.columns[-1], axis=1, inplace=True)

In [ ]:
sites = df.columns.get_level_values(0).unique()
sgs = df.columns.get_level_values(1).unique()

#Make now a plot where we compare the emissions of the different sites
# We group teh sg close to each site and each sg has the same color
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))
for j, site in enumerate(sites):
    serie = df.loc[4, site].astype(float)

    
    
    # Make bars for each sg
    for i, sg in enumerate(sgs):
        ax.bar(j*10 + i, serie[sg], color=f'C{i}', label=sg)

# Put the name of the sites as the label 
ax.set_xticks(np.arange(len(sites))*10 + 2.5)
ax.set_xticklabels(sites)


